In [ ]:
!huggingface-cli login

In [2]:
# !pip install -U -q trl bitsandbytes peft hf_xet tensorboard
!pip install  -U -q transformers==4.51.3 git+https://github.com/huggingface/trl.git datasets bitsandbytes peft qwen-vl-utils wandb accelerate
# Tested with transformers==4.47.0.dev0, trl==0.12.0.dev0, datasets==3.0.2, bitsandbytes==0.44.1, peft==0.13.2, qwen-vl-utils==0.0.8, wandb==0.18.5, accelerate==1.0.1
!pip install -q torch==2.4.1+cu121 torchvision==0.19.1+cu121 torchaudio==2.4.1+cu121 --extra-index-url https://download.pytorch.org/whl/cu121

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.2 M

In [3]:
from datasets import load_dataset

dataset_name = "avuong/vm1"

# Load Dataset
dataset = load_dataset(dataset_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

verified_reasoning_traces.json:   0%|          | 0.00/8.87M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/613 [00:00<?, ? examples/s]

In [ ]:
# ###############################################################
# NO NEED TO RESTART RUNTIME, OVERRIDDEN PACKAGES DO NOT MATTER.
# ###############################################################
# Clone just LFS pointers for Med-R1 models
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/yuxianglai117/Med-R1
# Pull only the Med-R1/VQA_CT model.
# This should take about 6 minutes with the basic Colab runtime.
!cd Med-R1/ && git lfs pull --include="VQA_CT"
# Pull the OmniMedVQA dataset.
!wget https://huggingface.co/datasets/foreverbeliever/OmniMedVQA/resolve/main/OmniMedVQA.zip
!unzip OmniMedVQA
!rm OmniMedVQA.zip
# Install packages + get JSON files.
!mkdir gh
!cd gh && git clone https://github.com/Yuxiang-Lai117/Med-R1.git
# !cd gh/Med-R1/src/r1-v && pip install -e ".[dev]"

# # Addtional modules
# !pip install wandb==0.18.3
# !pip install tensorboardx
# !pip install qwen_vl_utils torchvision
# !pip install flash-attn==2.8.2 --no-build-isolation

# # vLLM support
# !pip install vllm==0.8.5.post1

# # fix transformers version
# !pip install transformers==4.52.4

In [5]:
import torch
!pip3 install numpy
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig

model_id = "Med-R1/VQA_CT"

# Load model and tokenizer
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    #attn_implementation="flash_attention_2"
)
processor = AutoProcessor.from_pretrained(model_id)
processor.tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
from trl import SFTConfig

training_args = SFTConfig(
    output_dir="vm1-sft",                                           # Directory to save the model and push to the Hub. Use a specific repository id (e.g., gemma-3-4b-it-trl-sft-MMIU-Benchmark for multi-image datasets).
    max_seq_length=32768,
    num_train_epochs=5,                                             # Set the number of epochs to train the model.
    per_device_train_batch_size=1,                                  # Batch size for each device (e.g., GPU) during training. multi-image -> per_device_train_batch_size=1
    gradient_accumulation_steps=4,                                  # Number of steps before performing a backward/update pass to accumulate gradients. multi-image -> gradient_accumulation_steps=1
    gradient_checkpointing=True,                                    # Enable gradient checkpointing to reduce memory usage during training.
    logging_steps=10,                                               # Frequency of logging training progress (log every 10 steps).
    save_strategy="epoch",                                          # Save checkpoints at the end of each epoch.
    learning_rate=2e-05,                                            # Learning rate for training.
    bf16=True,                                                      # Enable bfloat16 precision for training to save memory and speed up computations.
    push_to_hub=True,                                               # Automatically push the fine-tuned model to Hugging Face Hub after training.
    gradient_checkpointing_kwargs={"use_reentrant": False},         # Set gradient checkpointing to non-reentrant to avoid issues.
    dataset_kwargs={"skip_prepare_dataset": True},                  # Skip dataset preparation to handle preprocessing manually.
    remove_unused_columns=False,                                    # Ensure unused columns are not removed in the collator (important for batch processing).
    #eos_token="<|im_end|>"
    #dataset_text_field="reasoning",
)

In [8]:
from trl import SFTTrainer
from PIL import Image
QUESTION_TEMPLATE = "{Question} First output the thinking process in <think> </think> and final choice (A, B, C, D ...) in <answer> </answer> tags."

def create_chat_prompt(example):
    return [
        {"role": "user", "content": [{"type": "image", "image": example["image"]}]},
        {"role": "user", "content": [{"type": "text", "text": QUESTION_TEMPLATE.format(Question=example["problem"])}]},
        {"role": "assistant", "content": [{"type": "text", "text": example["model_answer"]}]} # Include the target reasoning as the assistant's response
    ]


# https://huggingface.co/docs/trl/en/training_vlm_sft
def collate_fn(examples):

    msgs = [create_chat_prompt(example) for example in examples]
    texts = [processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True) for msg in msgs]
    images = [
        Image.open(f"OmniMedVQA/{example['image']}").convert("RGB")
        for example in examples
    ]

    # Tokenize the texts and process the images
    batch = processor(
        text=texts, images=images, return_tensors="pt", padding=True
    )  # Encode texts and images into tensors
    # The labels are the input_ids, and we mask the padding tokens in the loss computation
    labels = batch["input_ids"].clone()  # Clone input IDs for labels
    # Mask image tokens
    image_token_id = [
        processor.tokenizer.convert_tokens_to_ids(processor.tokenizer.special_tokens_map["additional_special_tokens"])
    ]
    start_of_image_token_id = processor.tokenizer.convert_tokens_to_ids("<|start_of_image|>")
    end_of_image_token_id = processor.tokenizer.convert_tokens_to_ids("<|end_of_image|>")
    image_token_id = processor.tokenizer.convert_tokens_to_ids("<|image|>")
    # Mask tokens for not being used in the loss computation
    labels[labels == processor.tokenizer.pad_token_id] = -100
    labels[labels == start_of_image_token_id] = -100
    labels[labels == end_of_image_token_id] = -100
    labels[labels == image_token_id] = -100

    batch["labels"] = labels
    return batch  # Return the prepared batch

training_args.eos_token = "<|im_end|>"
trainer = SFTTrainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=dataset["train"],
)

trainer.train()

# Save the final model
trainer.save_model()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: achvuong (achvuong-university-of-california) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Step,Training Loss
10,3.298200
20,2.255300
30,1.895600
40,1.504800
50,1.130800
60,1.005100
70,0.933900
80,0.917400
90,0.952800
100,0.882000


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/uti

Uploading...:   0%|          | 0.00/4.43G [00:00<?, ?B/s]